In [ ]:
from PyQt6.QtCore import Qt
print(Qt)


In [1]:
import numpy as np
import matplotlib
matplotlib.use('qtagg') 
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from scipy.integrate import RK45, DOP853, solve_ivp
import sympy as sp


### Equations:
[1] https://www.myphysicslab.com/pendulum/double-pendulum-en.html \
[2] https://www.jousefmurad.com/engineering/double-pendulum-1/ 

In [10]:
# Stałe fizyczne
przyspieszenie_ziemskie = 9.81  # Przyspieszenie ziemskie
dlugosc_wahadla_1 = 1.25         # Długość pierwszego wahadła
dlugosc_wahadla_2 = 2.0         # Długość drugiego wahadła
masa_wahadla_1 = 1.5            # Masa pierwszego wahadła
masa_wahadla_2 = 2.25           # Masa drugiego wahadła

# Parametry symulacji
kat_poczatkowy_1 = np.pi/2    # Kąt początkowy pierwszego wahadła
kat_poczatkowy_2 = np.pi        # Kąt początkowy drugiego wahadła
krok_czasowy = 0.0005           # Krok czasowy
czas_symulacji = 160             # Czas symulacji

# Funkcja lambda do obliczania Hamiltonianu
oblicz_hamiltonian = lambda theta1, theta2, w1, w2: (
    0.5 * masa_wahadla_1 * (dlugosc_wahadla_1 * w1) ** 2
    + 0.5
    * masa_wahadla_2
    * (
        (dlugosc_wahadla_1 * w1) ** 2
        + (dlugosc_wahadla_2 * w2) ** 2
        + 2
        * dlugosc_wahadla_1
        * dlugosc_wahadla_2
        * w1
        * w2
        * np.cos(theta1 - theta2)
    ),
    -masa_wahadla_1 * przyspieszenie_ziemskie * dlugosc_wahadla_1 * np.cos(theta1)
    - masa_wahadla_2
    * przyspieszenie_ziemskie
    * (dlugosc_wahadla_1 * np.cos(theta1) + dlugosc_wahadla_2 * np.cos(theta2)),
)

def pochodne(y, wspolczynnik_strat=0.0):
    """
    [1] Równania różniczkowe dla wahadła podwójnego zostały wyprowadzone
    przy użyciu równań Lagrange'a drugiego rodzaju. 
    """
    theta1, theta2, w1, w2 = y

    mianownik = (
        2 * masa_wahadla_1
        + masa_wahadla_2
        - masa_wahadla_2 * np.cos(2 * theta1 - 2 * theta2)
    )

    dtheta1 = w1
    dtheta2 = w2
    dw1 = (
        -przyspieszenie_ziemskie * (2 * masa_wahadla_1 + masa_wahadla_2) * np.sin(theta1)
        - masa_wahadla_2 * przyspieszenie_ziemskie * np.sin(theta1 - 2 * theta2)
        - 2
        * np.sin(theta1 - theta2)
        * masa_wahadla_2
        * (w2 * w2 * dlugosc_wahadla_2 + w1 * w1 * dlugosc_wahadla_1 * np.cos(theta1 - theta2))
    ) / (dlugosc_wahadla_1 * mianownik) - wspolczynnik_strat * w1  
    dw2 = (
        2
        * np.sin(theta1 - theta2)
        * (
            w1 * w1 * dlugosc_wahadla_1 * (masa_wahadla_1 + masa_wahadla_2)
            + przyspieszenie_ziemskie * (masa_wahadla_1 + masa_wahadla_2) * np.cos(theta1)
            + w2 * w2 * dlugosc_wahadla_2 * masa_wahadla_2 * np.cos(theta1 - theta2)
        )
    ) / (dlugosc_wahadla_2 * mianownik) - wspolczynnik_strat * w2  

    # Obliczenie energii całkowitej, kinetycznej i potencjalnej układu
    Kin, Vpot = oblicz_hamiltonian(theta1, theta2, w1, w2)
    Etot = Kin + Vpot
    return [dtheta1, dtheta2, dw1, dw2, Etot, Kin, Vpot]

# Funkcja lambda do obliczania pochodnych
pochodne_rk45 = lambda t, y: pochodne(y)[:4]


def oblicz_energie(wyniki):
    """Oblicza energie dla każdego kroku czasowego w symulacji."""
    energie_calkowite = []
    energie_kinetyczne = []
    energie_potencjalne = []
    for wynik in wyniki:
        theta1, theta2, w1, w2 = wynik
        Kin, Vpot = oblicz_hamiltonian(theta1, theta2, w1, w2)
        Etot = Kin + Vpot
        energie_calkowite.append(Etot)
        energie_kinetyczne.append(Kin)
        energie_potencjalne.append(Vpot)
    return (
        np.array(energie_calkowite),
        np.array(energie_kinetyczne),
        np.array(energie_potencjalne),
    )
    
def symulacja(
    kat_poczatkowy_1,
    kat_poczatkowy_2,
    krok_czasowy,
    czas_symulacji,
    wspolczynnik_strat=0.0,
):
    """Symulacja wahadła dwuciałowego."""

    # Warunki początkowe
    warunki_poczatkowe = np.array([kat_poczatkowy_1, kat_poczatkowy_2, 0.0, 0.0])
    
    pochodne_rk45 = lambda t, y: pochodne(y, wspolczynnik_strat)[:4]

    # Rozwiązanie układu równań różniczkowych
    rk45 = DOP853(
        pochodne_rk45, 0, warunki_poczatkowe, czas_symulacji, max_step=krok_czasowy,
        rtol = 1e-6, atol=1e-08
    )
    czasy = []
    wyniki = []
    while rk45.status == "running":
        rk45.step()
        czasy.append(rk45.t)
        wyniki.append(rk45.y)
    wyniki = np.array(wyniki)

    # Obliczenie energii dla każdego kroku czasowego
    Etot, Kin, Vpot = oblicz_energie(wyniki)

    # Dodanie energii do tablicy wyniki
    wyniki = np.column_stack((wyniki, Etot, Kin, Vpot))

    return wyniki, Etot, Kin, Vpot, czasy

def animacja(wyniki, energie_calkowite, energie_kinetyczne, energie_potencjalne, czasy, krok_czasowy):
    """Tworzy animację wahadła."""

    theta1, theta2 = wyniki[:, 0], wyniki[:, 1]

    # Współrzędne x, y mas
    x1 = dlugosc_wahadla_1 * np.sin(theta1)
    y1 = -dlugosc_wahadla_1 * np.cos(theta1)
    x2 = x1 + dlugosc_wahadla_2 * np.sin(theta2)
    y2 = y1 - dlugosc_wahadla_2 * np.cos(theta2)

    # Ustawienia wykresu
    fig = plt.figure(figsize=(16, 8))
    plt.tight_layout()
    ax1 = fig.add_subplot(121, autoscale_on=False, xlim=(-np.max(dlugosc_wahadla_1+dlugosc_wahadla_2),
                                                        np.max(dlugosc_wahadla_1+dlugosc_wahadla_2)),
                                                    ylim=(-np.max(dlugosc_wahadla_1+dlugosc_wahadla_2),
                                                          np.max(dlugosc_wahadla_1+dlugosc_wahadla_2)))
    ax1.set_aspect("equal")
    ax1.grid()

    # Obliczenie minimalnej i maksymalnej wartości dla ylim
    ylim_min = np.min(
        [
            np.min(energie_calkowite),
            np.min(energie_kinetyczne),
            np.min(energie_potencjalne),
        ]
    )
    ylim_max = np.max(
        [
            np.max(energie_calkowite),
            np.max(energie_kinetyczne),
            np.max(energie_potencjalne),
        ]
    )

    ax2 = fig.add_subplot(
        122, xlim=(0, czasy[-1]), ylim=(ylim_min, ylim_max)
    )  
    ax2.set_xlabel("Czas (s)")
    ax2.set_ylabel("Energia (J)")
    ax2.grid()

    # Elementy animacji
    (linia,) = ax1.plot([], [], "o-", lw=2)
    szablon_czasu = "czas = %.1fs"
    tekst_czasu = ax1.text(0.05, 0.9, "", transform=ax1.transAxes)
    (linia_E,) = ax2.plot([], [], "k-", lw=2, label="E")
    (linia_T,) = ax2.plot([], [], "r-", lw=2, label="T")
    (linia_V,) = ax2.plot([], [], "b-", lw=2, label="V")
    ax2.legend()

    def init():
        linia.set_data([], [])
        tekst_czasu.set_text("")
        linia_E.set_data([], [])
        linia_T.set_data([], [])
        linia_V.set_data([], [])
        return linia, tekst_czasu, linia_E, linia_T, linia_V

    # Wstępne obliczenie współrzędnych
    x = np.array([
        [0] * len(wyniki),
        dlugosc_wahadla_1 * np.sin(wyniki[:, 0]),
        dlugosc_wahadla_1 * np.sin(wyniki[:, 0]) + dlugosc_wahadla_2 * np.sin(wyniki[:, 1])
    ]).T
    y = np.array([
        [0] * len(wyniki),
        -dlugosc_wahadla_1 * np.cos(wyniki[:, 0]),
        -dlugosc_wahadla_1 * np.cos(wyniki[:, 0]) - dlugosc_wahadla_2 * np.cos(wyniki[:, 1])
    ]).T

    def animate(i):
        linia.set_data(x[i], y[i])
        tekst_czasu.set_text(szablon_czasu % (czasy[i]))
        # Aktualizacja danych linii dla energii co 10 klatek
        if i % 10 == 0:
            linia_E.set_data(czasy[:i], energie_calkowite[:i])
            linia_T.set_data(czasy[:i], energie_kinetyczne[:i])
            linia_V.set_data(czasy[:i], energie_potencjalne[:i])
        return linia, tekst_czasu, linia_E, linia_T, linia_V

    ani = FuncAnimation(
        fig,
        animate,
        np.arange(1, len(wyniki)),
        interval=krok_czasowy*100,
        blit=True,
        init_func=init,
    )

    # Wyświetlenie wykresu energii w czasie
    plt.figure()
    plt.plot(czasy, energie_calkowite, "k-", lw=2, label="E")
    plt.plot(czasy, energie_kinetyczne, "r-", lw=2, label="T")
    plt.plot(czasy, energie_potencjalne, "b-", lw=2, label="V")
    plt.xlabel("Czas (s)")
    plt.ylabel("Energia (J)")
    plt.title("Energia w czasie")
    plt.legend()
    plt.show()

    return ani

# Symulacja
(
    wyniki,
    energie_calkowite,
    energie_kinetyczne,
    energie_potencjalne,
    czasy,
) = symulacja(
    kat_poczatkowy_1,
    kat_poczatkowy_2,
    krok_czasowy,
    czas_symulacji,
    wspolczynnik_strat=0.005
)

# Animacja
animacja = animacja(
    wyniki,
    energie_calkowite,
    energie_kinetyczne,
    energie_potencjalne,
    czasy,
    krok_czasowy,
)


### V2 Analityczne rozwiązanie równań razem z współczynnikiem straty

In [ ]:
# Definicja symboli
m1, m2, l1, l2, g, theta1, theta2, t = sp.symbols('m1 m2 l1 l2 g theta1 theta2 t')
k1, k2, f1, f2, alpha, beta = sp.symbols('k1 k2 f1 f2 alpha beta')

# Definicja funkcji theta1(t) i theta2(t)
theta1 = sp.Function('theta1')(t)
theta2 = sp.Function('theta2')(t)
# Definicja omega_1 i omega_2
omega_1 = sp.Function('omega_1')(t)
omega_2 = sp.Function('omega_2')(t)

# Pochodne
dtheta1_dt = sp.diff(theta1, t)
dtheta2_dt = sp.diff(theta2, t)
d2theta1_dt2 = sp.diff(theta1, t, t)
d2theta2_dt2 = sp.diff(theta2, t, t)
# Zmienne pomocnicze
cos_diff = sp.cos(theta2 - theta1)
sin_diff = sp.sin(theta2 - theta1)
glm      = g*l1*m1
# Definicja alpha i beta
alpha = k1 * dtheta1_dt - f1 * sp.cos(omega_1 * t)  
beta  = k2 * dtheta2_dt - f2 * sp.cos(omega_2 * t)  

# Równania z uwzględnieniem strat energii (z użyciem zmiennych pomocniczych)
eq1 = sp.Eq((m1 + m2) * l1**2 * d2theta1_dt2 + m2 * l1 * l2 * d2theta2_dt2 * cos_diff
            - m2 * l1 * l2 * (dtheta2_dt)**2 * sin_diff + (m1 + m2) * g * l1 * sp.sin(theta1)
            + alpha
            , 0 )
eq2 = sp.Eq(m2 * l2**2 * d2theta2_dt2 + m2 * l1 * l2 * d2theta1_dt2 * cos_diff
            + m2 * l1 * l2 * (dtheta1_dt)**2 * sin_diff + m2 * g * l2 * sp.sin(theta2)
            + beta
            , 0 )

#Uproszczenie równań - kombinacja funkcji
eq1_simplified = sp.expand_trig(eq1)          # Rozwinięcie funkcji trygonometrycznych
eq2_simplified = sp.expand_trig(eq2)          
eq1_simplified = sp.factor(eq1_simplified)    # Wyciągnięcie czynnikiów wspólne
eq2_simplified = sp.factor(eq2_simplified)    
eq1_simplified = sp.trigsimp(eq1_simplified)  # Uproszczenie wyrażeń trygonometrycznych
eq2_simplified = sp.trigsimp(eq2_simplified) 
eq1_simplified = sp.simplify(eq1_simplified)  # Ogólne uproszczenie
eq2_simplified = sp.simplify(eq2_simplified)  


# Rozwiązanie równań
solution = sp.solve([eq1_simplified, eq2_simplified], (sp.diff(theta1, t, t), sp.diff(theta2, t, t)))

# Przypisanie rozwiązań do zmiennych
d2theta1_dt2_sol = solution[sp.diff(theta1, t, t)]
d2theta2_dt2_sol = solution[sp.diff(theta2, t, t)]

# Uproszczenie rozwiązań
d2theta1_dt2_simplified = sp.expand_trig(d2theta1_dt2_sol)      # Rozwinięcie funkcji trygonometrycznych
d2theta2_dt2_simplified = sp.expand_trig(d2theta2_dt2_sol)      
d2theta1_dt2_simplified = sp.factor(d2theta1_dt2_simplified)    # Wyciągnięcie czynnikiów wspólne
d2theta2_dt2_simplified = sp.factor(d2theta2_dt2_simplified)    
d2theta1_dt2_simplified = sp.trigsimp(d2theta1_dt2_simplified)  # Uproszczenie wyrażeń trygonometrycznych
d2theta2_dt2_simplified = sp.trigsimp(d2theta2_dt2_simplified)  
d2theta1_dt2_simplified = sp.simplify(d2theta1_dt2_simplified, deep=True)  # Ogólne uproszczenie
d2theta2_dt2_simplified = sp.simplify(d2theta2_dt2_simplified, deep=True)  


Równanie dla domega_1_dt:
                                                                               ↪
d           2⋅f₁⋅l₂⋅cos(t⋅ω₁(t)) - f₂⋅l₁⋅cos(t⋅ω₂(t) - θ₁(t) + θ₂(t)) - f₂⋅l₁⋅ ↪
──(ω₁(t)) = ────────────────────────────────────────────────────────────────── ↪
dt                                                                             ↪
                                                                               ↪

↪                                                                              ↪
↪ cos(t⋅ω₂(t) + θ₁(t) - θ₂(t)) - 2⋅g⋅l₁⋅l₂⋅m₁⋅sin(θ₁(t)) - g⋅l₁⋅l₂⋅m₂⋅sin(θ₁(t ↪
↪ ──────────────────────────────────────────────────────────────────────────── ↪
↪                                                                     2        ↪
↪                                                                   l₁ ⋅l₂⋅(2⋅ ↪

↪                                                                              ↪
↪ ) - 2⋅θ₂(t)) - g⋅l₁⋅l₂⋅m₂⋅sin(θ₁(t)) - 2⋅k₁⋅l₂⋅ω₁(t) + 2⋅k₂⋅l₁⋅ω₂(t)⋅cos(θ₁( ↪

In [33]:
d2theta1_dt2_simplified

(2*f1*l2*cos(omega_1*t) - f2*l1*cos(omega_2*t - theta1(t) + theta2(t)) - f2*l1*cos(omega_2*t + theta1(t) - theta2(t)) - 2*g*l1*l2*m1*sin(theta1(t)) - g*l1*l2*m2*sin(theta1(t) - 2*theta2(t)) - g*l1*l2*m2*sin(theta1(t)) - 2*k1*l2*Derivative(theta1(t), t) + 2*k2*l1*cos(theta1(t) - theta2(t))*Derivative(theta2(t), t) - l1**2*l2*m2*sin(2*theta1(t) - 2*theta2(t))*Derivative(theta1(t), t)**2 - 2*l1*l2**2*m2*sin(theta1(t) - theta2(t))*Derivative(theta2(t), t)**2)/(l1**2*l2*(2*m1 - m2*cos(2*theta1(t) - 2*theta2(t)) + m2))

In [6]:
d2theta2_dt2_simplified

(-f1*l2*m2*cos(omega_1*t - theta1(t) + theta2(t)) - f1*l2*m2*cos(omega_1*t + theta1(t) - theta2(t)) + 2*f2*l1*m1*cos(omega_2*t) + 2*f2*l1*m2*cos(omega_2*t) + g*l1*l2*m1*m2*sin(2*theta1(t) - theta2(t)) - g*l1*l2*m1*m2*sin(theta2(t)) + g*l1*l2*m2**2*sin(2*theta1(t) - theta2(t)) - g*l1*l2*m2**2*sin(theta2(t)) + 2*k1*l2*m2*cos(theta1(t) - theta2(t))*Derivative(theta1(t), t) - 2*k2*l1*m1*Derivative(theta2(t), t) - 2*k2*l1*m2*Derivative(theta2(t), t) + 2*l1**2*l2*m1*m2*sin(theta1(t) - theta2(t))*Derivative(theta1(t), t)**2 + 2*l1**2*l2*m2**2*sin(theta1(t) - theta2(t))*Derivative(theta1(t), t)**2 + l1*l2**2*m2**2*sin(2*theta1(t) - 2*theta2(t))*Derivative(theta2(t), t)**2)/(l1*l2**2*m2*(2*m1 - m2*cos(2*theta1(t) - 2*theta2(t)) + m2))

In [22]:

# Parametry symulacji
theta1_0 = np.pi / 2  # Kąt początkowy pierwszego wahadła
theta2_0 = np.pi  # Kąt początkowy drugiego wahadła
dt = 0.0005  # Krok czasowy
t_max = 160  # Czas symulacji

# Warunki początkowe
y0 = [theta1_0, theta2_0, 0.0, 0.0] 

# Kompilacja wyrażeń SymPy z jawnym określeniem modułów
d2theta1_dt2_func = sp.lambdify((theta1, theta2, dtheta1_dt, dtheta2_dt, m1, m2, l1, l2, g, k1, k2, f1, f2, omega_1, omega_2 ),
                                 d2theta1_dt2_simplified, modules=['numpy' , {'sin': np.sin, 'cos': np.cos}]) # , {'sin': np.sin, 'cos': np.cos}
d2theta2_dt2_func = sp.lambdify((theta1, theta2, dtheta1_dt, dtheta2_dt, m1, m2, l1, l2, g, k1, k2, f1, f2, omega_1, omega_2 ),
                                 d2theta2_dt2_simplified, modules=['numpy', {'sin': np.sin, 'cos': np.cos}])

print(type(theta1), type(theta2), type(dtheta1_dt), type(dtheta2_dt), type(m1), type(m2), type(l1), type(l2), type(g), type(omega_1), type(omega_1))

# def double_pendulum_equations(t, y, m1, m2, l1, l2, g, k1, k2, f1, f2, omega_1, omega_2 ):
#     """
#     Równania ruchu wahadła podwójnego z uwzględnieniem strat energii.
#     """
#     theta1_val, theta2_val, dtheta1_dt, dtheta2_dt = y  # Zmienione nazwy zmiennych

#     d2theta1_dt2_num = d2theta1_dt2_func(theta1_val, theta2_val, dtheta1_dt, dtheta2_dt, m1, m2, l1, l2, g, k1, k2, f1, f2, omega_1, omega_2 )
#     d2theta2_dt2_num = d2theta2_dt2_func(theta1_val, theta2_val, dtheta1_dt, dtheta2_dt, m1, m2, l1, l2, g, k1, k2, f1, f2, omega_1, omega_2 )
#     return [dtheta1_dt, dtheta2_dt, d2theta1_dt2_num, d2theta2_dt2_num]


# # Stałe fizyczne
# g = 9.81  # Przyspieszenie ziemskie
# l1 = 1.25  # Długość pierwszego wahadła
# l2 = 2.0  # Długość drugiego wahadła
# m1 = 1.5  # Masa pierwszego wahadła
# m2 = 2.25  # Masa drugiego wahadła
# k1 = 0.01  # Współczynnik tłumienia dla pierwszego wahadła
# k2 = 0.01  # Współczynnik tłumienia dla drugiego wahadła
# f1 = 0.01
# f2 = 0.01
# omega_1 = 0.01
# omega_2 = 0.01


# # Zakres czasu
# t_span = (0, t_max)
# t_eval = np.arange(t_span[0], t_span[1], dt)

# # Rozwiązanie numeryczne
# sol = solve_ivp(double_pendulum_equations, t_span, y0, method='DOP853',
#                 t_eval=t_eval, args=(m1, m2, l1, l2, g, k1, k2, f1, f2, omega_1, omega_2))

# # Wyniki
# theta1_num = sol.y[0]
# theta2_num = sol.y[1]
# t_num = sol.t

# def animacja(theta1_num, theta2_num, t_num, l1, l2, dt):
#     """Tworzy animację wahadła."""
    
#     # Współrzędne x, y mas
#     x1 = l1 * np.sin(theta1_num)
#     y1 = -l1 * np.cos(theta1_num)
#     x2 = x1 + l2 * np.sin(theta2_num)
#     y2 = y1 - l2 * np.cos(theta2_num)

#     # Ustawienia wykresu
#     fig = plt.figure()
#     ax = fig.add_subplot(111, autoscale_on=False, xlim=(-l1-l2, l1+l2), ylim=(-l1-l2, l1+l2))
#     ax.set_aspect('equal')
#     ax.grid()

#     # Elementy animacji
#     (linia,) = ax.plot([], [], 'o-', lw=2)
#     szablon_czasu = 'czas = %.1fs'
#     tekst_czasu = ax.text(0.05, 0.9, '', transform=ax.transAxes)

#     def init():
#         linia.set_data([], [])
#         tekst_czasu.set_text('')
#         return linia, tekst_czasu

#     def animate(i):
#         thisx = [0, x1[i], x2[i]]
#         thisy = [0, y1[i], y2[i]]

#         linia.set_data(thisx, thisy)
#         tekst_czasu.set_text(szablon_czasu % (t_num[i]))
#         return linia, tekst_czasu

#     ani = FuncAnimation(fig, animate, np.arange(1, len(t_num)),
#                         interval=dt, blit=True, init_func=init)
#     plt.show()

#     return ani

# animacja = animacja(theta1_num, theta2_num, t_num, l1, l2, dt)

theta1 theta2 <class 'sympy.core.function.Derivative'> <class 'sympy.core.function.Derivative'> <class 'sympy.core.symbol.Symbol'> <class 'sympy.core.symbol.Symbol'> <class 'sympy.core.symbol.Symbol'> <class 'sympy.core.symbol.Symbol'> <class 'sympy.core.symbol.Symbol'> <class 'sympy.core.symbol.Symbol'> <class 'sympy.core.symbol.Symbol'>


In [ ]:
sol